# Summary

Here we test the tools developped in Utilities/Optobot_utils.py. We then use these tools to analyse Irene's optobot experiments.

# Package imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
sys.path.insert(0, "../Utilities")

# sys.path.append("/home/durrieu/Tracking_Analysis/Utilities")
# sys.path.append("/Users/ulric/git/Tracking_Analysis/Utilities")
import Utils
import Optobot_utils

# Accessing the experiments

We first developp a function to find the experiments in the main directory. We then use a custom class to load the experiments, including metadata and DLC data.

In [ ]:
Exps = Optobot_utils.find_experiments(
    Utils.get_labserver() / "Experimental_data" / "Irene_Optobot"
)

Exps

In [ ]:
TestFly = Exps[0]

In [ ]:
# Extract the npy file called experiment_dict.npy

# Load the experiment_dict.npy file
exp_dict = np.load(TestFly / "experiment_dict.npy", allow_pickle=True).item()

exp_dict

In [ ]:
exp_dict["fly0"]

In [ ]:
# Load the genotype_dict.npy file
gen_dict = np.load(TestFly.parent.parent / "genotype_dict.npy", allow_pickle=True).item()

gen_dict

In [ ]:
# Find a .pkl file in the folder
pkl_files = list(TestFly.glob("*.pkl"))

# Get the first .pkl file
pkl_file = pkl_files[0]

In [ ]:
# Load the .pkl file
import pickle
with open(pkl_file, "rb") as input:
    data = pickle.load(input)

In [ ]:
import pandas as pd

# Read the .pkl file
data = pd.read_pickle(pkl_file)

data

In [ ]:
# Solve multiindex

data.columns = data.columns.droplevel(0)

data.reset_index(inplace=True)

data

In [ ]:
print(data.columns)

In [ ]:
# Drop the top level of the multi-index for 'pos_x' and 'pos_y'
data.columns = (
    data.columns.droplevel(0)
    if "pos_x" in data.columns or "pos_y" in data.columns
    else data.columns
)

data

# Testing dataframe loading

In [ ]:
TestFly = Optobot_utils.Fly(Exps[0])

In [ ]:
TestFly.data

In [ ]:
Exps[0].parent.parent